In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [2]:
chromedriver = '/Users/hhlee/Desktop/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

In [3]:
base_url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query='
url = base_url + quote('강아지')

In [4]:
driver.get(url)

In [5]:
# blog 페이지를 아래로 내려서 많은 데이터 확보
from selenium.webdriver.common.keys import Keys

body = driver.find_element_by_tag_name('body')
for i in range(30):
     body.send_keys(Keys.PAGE_DOWN)
     time.sleep(2)


In [6]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [7]:
lis = soup.select('li.bx._svp_item')
len(lis)

61

In [8]:
li = lis[0]
title = li.select_one('a.api_txt_lines.total_tit._cross_trigger').get_text()
content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
date = li.select_one('.sub_time.sub_txt').get_text()
title, date, content

('강아지가 너무 저만 좋아해서 난감해요..',
 '5일 전',
 '저는 재택근무라 24시간 집에 있고 신랑은 저녁에 퇴근하고 강아지랑 놀아줘요. 일단 주보호자는 저예요.... 무슨 왕따시키는것도 아니고 말이에요..ㅠㅠ 강아지가 이럴때마다 신랑이 너무 서운해해서 괜히 눈치보게 돼요.....')

In [9]:
title_list, content_list, date_list = [],[],[]
for i, li in enumerate(lis):
    try:
         adv = li.select_one('a.link_ad').get_text()
         if adv.strip() == '광고':
                continue
    except:


         title = li.select_one('a.api_txt_lines.total_tit').get_text()
         content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
         date = li.select_one('.sub_time.sub_txt').get_text()

         title_list.append(title)
         content_list.append(content)
         date_list.append(date)

         #print(i)

In [10]:
df = pd.DataFrame({
        'title': title_list, 'date': date_list, 'content': content_list
})
df.head()

,title,date,content
0,강아지가 너무 저만 좋아해서 난감해요..,5일 전,저는 재택근무라 24시간 집에 있고 신랑은 저녁에 퇴근하고 강아지랑 놀아줘요. 일단...
1,저는 강아지를 방치하고있는 견주입니다.,4일 전,생각해서 강아지의 인생은 신경도 안쓰고 강사모에서 둥둥이를 입양하게 되었습니다. 너...
2,수원동물병원 강아지 유선종양 수술 받았어요~,7시간 전,증상에 대해 검색해보니 강아지 유선종양과 비슷해서 전문적인 치료를 받을 수 있는 곳...
3,강아지사료 닥터독 시니어 건강하개!,2021.08.23.,오랜만에 소개할 제품은 닥터독 #강아지사료 시니어입니다. 쿤이도 한때는 이 사료 저...
4,간종양 때문에 죽다 살아난 10살 강아지,2021.07.29.,죽어가던 10살 노령견 강아지가 다시 살아날 수 있었던 이유 24시 우리들동물메디컬...


In [11]:
df.shape

(61, 3)

In [12]:
df.to_csv('data/puppy_blog.txt')

In [13]:
driver.close()